# Comparing Original LeetCode Solutions to Online Solutions

Leetcode is an ubiquitous platform used by those in the tech industry to prepare for data structures and algorithms problems that may arise during the interview process. In this notebook, we compare a dataset of originally developed Leetcode solutions (solved by myself over the span of a few years) to a dataset of solutions scraped from a set of leetcode.com forum pages, where one forum page is dedicated to each problem. The analysis is done using a series of techniques from Natural Language Processing. An list of the analyses we perform is below:

(Under development)

Ultimately, the goal of this project is to discover my strengths and weaknesses (with regards to data structures and algorithms, but also coding more generally) with a data driven methodology and use these to identify personal development areas.

## Part I: Scraping dataset of online solutions

Leetcode is a dynamic website built using React. For this reason, lightweight scrapers such as requests + beautifulsoup would not suffice. Instead, we use Selenium to scrape our database of solutions. This is done in 3 parts:

1. Scrape a list of all problems and slugs so that we may crawl individual problem pages.
2. For each problem, scrape the links of the top 4 pages of posts with the 'python3' tag on the Solutions forum.
3. Scrape the text for all post links scraped in step 2.

In [59]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from collections import defaultdict
import pandas as pd
import time
import re

pd.set_option('display.max_colwidth', None)

dataset_path = 'dataset'

In [104]:
def get_premium_info(el):
        '''
        Searches a specific div element given as argument for an orange svg object, which indicates the bounding box for the leetcode premium lock symbol on the left of a problem.
        This svg is only present for premium problems.
        '''
        try:
            el.find_element(By.CLASS_NAME, 'text-brand-orange')
        except NoSuchElementException:
            return False
        return True
    
def scrape_problems_list(num_pages= 50, problems_per_page=50):
    '''
    Scrapes master list of problems on leetcode in order to get a dataset of hyperlinks (or equivalently, slugs) to the page dedicated to each individual problem.
    '''
    problems_dict = defaultdict(list)
    for i in range(num_pages):
        print(f"Page: {i+1}")
        source_page = f'https://leetcode.com/problemset/all/?page={i+1}'
        driver = webdriver.Chrome()
        driver.get(source_page)
        time.sleep(60) if (i and not i % 10) else time.sleep(5) # Allow page to load without implicitly_wait (because we check for premium by checking for exception)
        all_rows = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[6]/div[2]/div/div/div[2]')
        j = 0
        while j < problems_per_page:
            try:
                problem_row = all_rows.find_element(By.XPATH, f'./div[{j + 1}]')
                elements = problem_row.find_elements(By.CLASS_NAME, 'mx-2')
                problems_dict['premium'].append(get_premium_info(elements[0]))
                problems_dict['href'].append(elements[1].find_element(By.XPATH, './/div/div/div/div/a').get_attribute('href'))
                problems_dict['title'].append(elements[1].find_element(By.XPATH, './/div/div/div/div/a').text)
                problems_dict['acceptance'].append(elements[3].find_element(By.XPATH, './/span').text)
                problems_dict['difficulty'].append(elements[4].find_element(By.XPATH, './/span').text)
            except NoSuchElementException:
                print(f'Fewer than {problems_per_page} problems on this page.')
            j += 1
    return pd.DataFrame(problems_dict)

def scrape_solutions_forum(problem_slug, num_pages=4):
    '''
    Scrapes the solutions forum dedicated to a given problem (specified via problem slug) in order get a dataset of hyperlinks to the top 60 posts (by votes) with the 'python3' tag for that problem.
    '''
    post_dict = defaultdict(list)
    for i in range(1, num_pages + 1):
        source_page = f'https://leetcode.com/problems/{problem_slug}/discuss/?currentPage={i}&orderBy=most_votes&query=&tag=python3'
        driver = webdriver.Chrome()
        driver.get(source_page)
        driver.implicitly_wait(5) # Allow page to load before throwing exception
        current_page_posts = driver.find_elements(By.CLASS_NAME, 'topic-item-wrap__2FSZ')
        for post in current_page_posts:
            href = post.find_element(By.XPATH, './/*[@class="topic-item__1Asc"]/div[1]/div[1]/a').get_attribute('href')
            title = post.find_element(By.XPATH, './/*[@class="topic-item__1Asc"]/div[1]/div[1]/a/div/div').text
            try:
                user = post.find_element(By.XPATH, './/*[@class="topic-item__1Asc"]/div[1]/div[2]/span/span[1]/a').text
            except NoSuchElementException:
                user = 'deleted_user'
            upvotes = post.find_element(By.XPATH, './/*[@class="topic-item__1Asc"]/div[2]/div[1]/div').text
            views = post.find_element(By.XPATH, './/*[@class="topic-item__1Asc"]/div[2]/div[2]/div').text
            post_dict['slug'].append(problem_slug)
            post_dict['title'].append(title)
            post_dict['user'].append(user)
            post_dict['upvotes'].append(upvotes)
            post_dict['views'].append(views)
            post_dict['href'].append(href)    
        time.sleep(2)
    return pd.DataFrame(post_dict)

def parse_html_to_python(html_code_block):
    lines = html_code_block.split('\n')
    solution_flag = True
    python_flag = True
    if '<span class="hljs-class">' not in lines[0]:
        solution_flag = False
    res = [re.sub('</span>', '', re.sub('<span class="hljs-[\s\S]*?">', '', line)) for line in lines] # for code formatting / coloring
    res = [re.sub('&gt;', '>', re.sub('&lt;', '<', line)) for line in res] # HTML entities
    res = '\n'.join(res)
    # Some posts contain solutions for multiple languages. This is a nifty way to check for python solutions, 
    # since every solution defines a function and no other language requires "(self," in function declarations within a class.
    if '(self,' not in res:
        python_flag = False
    return solution_flag and python_flag, res

def scrape_single_post(post_href):
    '''
    Scrapes a single post to return the text written in the original post. Comments are ignored.
    '''
    driver = webdriver.Chrome()
    driver.get(post_href)
    driver.implicitly_wait(5)
    original_post_body = driver.find_elements(By.XPATH, '//*[@id="discuss-container"]/div/div/div[2]/div[1]/div[2]/div[2]/div')[0]
    try:
        code_blocks = original_post_body.find_elements(By.XPATH, './/pre') # <pre> </pre> used for code formatting
    except NoSuchElementException:
        pass # No code blocks in the original post
    all_solutions_in_post = []
    for block in code_blocks:
        html = block.find_element(By.XPATH, './/code').get_attribute('innerHTML')
        valid_solution, parsed_html = parse_html_to_python(html)
        if valid_solution:
            all_solutions_in_post.append(parsed_html)
    return all_solutions_in_post

Scrape problems list and preprocess for a list of all problem names, sligs, numbers, acceptance rates, and difficulties. 

In [40]:
#problems_df = scrape_problems_list() # Uncomment to rescrape data
#problems_df.to_csv(f'{dataset_path}/problems_data.csv')
problems_df = pd.read_csv(f'{dataset_path}/problems_data.csv', index_col='Unnamed: 0')
problems_df['number'] = problems_df['title'].str.split('.').map(lambda x: x[0]).astype('int64')
problems_df['title_text'] = problems_df['title'].str.split('.').map(lambda x: x[1]).str[1:]
problems_df['slug'] = problems_df['href'].str.split('/').map(lambda x: x[-2])
problems_df['acceptance'] = problems_df['acceptance'].str[:-1].astype('float64') / 100. # Convert percentage to decimal

problems_df.drop_duplicates(inplace=True) # The featured problem will always be duplicated (once at top of list, once in proper numeric ordering)
problems_df.loc[problems_df['premium'] == 0] # Unable to scrape paid problems.
problems_df = problems_df[['title_text', 'slug', 'number', 'acceptance', 'difficulty']].sort_values('number').reset_index(drop=True)
problems_df.to_csv(f'{dataset_path}/problems_data_cleaned.csv')

Loop over problem slugs, saving the first 4 pages of posts (order by most views) with the 'python3' tag. 

In [ ]:
problems_df = pd.read_csv(f'{dataset_path}/problems_data_cleaned.csv', index_col='Unnamed: 0')
all_posts_hrefs = []
for i, slug in enumerate(problems_df['slug']):
    df_of_post_hrefs = scrape_solutions_forum(slug)
    all_posts_hrefs.append(df_of_post_hrefs)
    if i and not i % 10:
        temp_save_df = pd.concat(all_posts_hrefs, axis=0)
        temp_save_df.to_csv(f'{dataset_path}/post_hrefs/intermediate_post_hrefs_df_{i}.csv')
        time.sleep(60)
    else:
        time.sleep(5)
all_posts_hrefs_df = pd.concat(all_posts_hrefs, axis=0)
all_posts_hrefs.to_csv(f'{dataset_path}/all_posts_hrefs.csv')